In [1]:
import os
import getpass

from llama_index.llms.groq import Groq
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage)

from llama_index.core.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
from llama_index.tools.google import GoogleSearchToolSpec

In [ ]:
# 1. set up the embedding model: from hf
# 2. set up the llm (free :))
# 3. create and store the index
# 4. create a query engine
# 5. define a booksearch tool (we can add movie search tool, spotify album tool, etc)
# 6. define a search tool
# 7. Run query



### refererence:

Introduction to LlamaIndex with Python (2024) videos 1-3: https://www.youtube.com/watch?v=cCyYGYyCka4

https://docs.llamaindex.ai/en/stable/use_cases/agents/

tools:

https://llamahub.ai/?tab=tools

https://llamahub.ai/l/tools/llama-index-tools-google?from=tools

### 1. set up the embedding model: from hf

In [2]:
Settings.embed_model = HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5')

### 2. set up the llm (free :))

In [3]:
os.environ['GROQ_API_KEY'] = getpass.getpass("GROQ API KEY: ")

GROQ API KEY:  ········


In [4]:
# configure the LLM for querying
Settings.llm = Groq(model="llama3-70b-8192")

### 3. create and store the index

In [5]:
# Define the directory for storing the index
PERSIST_DIR = "./storage"

# Load or create the index
if not os.path.exists(PERSIST_DIR):
    # Load the documents and create the index, all the documents are saved in the data folder
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    
    # Store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # Load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

### 4. create a query engine

In [6]:
query_engine = index.as_query_engine()

### 5. define a booksearch tool (we can add movie search tool, spotify album tool, etc)

In [7]:
booksearch_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="best_books",
    description="A RAG engine with some information of popular books."
)

### 6. define a search tool

In [8]:
# # Import and initialize our tool spec
from llama_index.core.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
from llama_index.tools.google import GoogleSearchToolSpec

# reference: https://llamahub.ai/l/tools/llama-index-tools-google?from=tools
google_spec = GoogleSearchToolSpec(key="your-key", engine="your-engine-id")

tools = LoadAndSearchToolSpec.from_defaults(
    google_spec.to_tool_list()[0]
).to_tool_list()

all_tools = tools + [booksearch_tool]

# Initialize the ReAct Agent
agent = ReActAgent.from_tools(all_tools, verbose=True)

### 7. Run query

#### From the provided database

In [14]:
response1 = agent.chat("What is the best book to recommend to a 7 year old girl, and why?")

> Running step 599a8c9d-ca2e-41f9-8b4d-c59aa7532231. Step input: What is the best book to recommend to a 7 year old girl, and why?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: best_books
Action Input: {'input': 'What is the best book to recommend to a 7 year old girl?'}
Observation: Based on the provided data, I would recommend "Cinderella Ate My Daughter: Dispatches from the Frontlines of the New Girlie-Girl Culture" by Peggy Orenstein. Although the book's title might seem appealing to a 7-year-old girl, its content is more suitable for parents or adults who want to understand the modern girl culture. 

A more suitable recommendation for a 7-year-old girl would be "Deceptively Delicious: Simple Secrets to Get Your Kids Eating Good Food" by Jessica Seinfeld, which seems to be a children's book with a focus on healthy eating. However, please note that the book's content and reading level might not be suitable for a 7

#### from google search

In [16]:
response2 = agent.chat("What're the best things to do in Orland, FL?")

> Running step 4b8efa04-9942-4f0c-b4ba-a50467e70544. Step input: What're the best things to do in Orland, FL?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: google_search
Action Input: {'query': 'best things to do in orlando fl'}
Observation: Content loaded! You can now search the information using read_google_search
> Running step 49785247-d417-475e-826b-7ab2f15ca6ee. Step input: None
Thought: I have the search results from Google, now I need to read the relevant information to answer the user's question.
Action: read_google_search
Action Input: {'query': 'what are the top attractions in orlando florida'}
Observation: Based on the provided context, some top attractions in Orlando, Florida include water parks, thrill rides, and wildlife experiences. Additionally, there are indoor thrills, fun parks, activity centers, and more. Some specific attractions mentioned include Fun Spot America, which features go-kart tracks 